In [1]:
# importamos librerías
import pandas as pd
from datetime import datetime

# **Carga del archivo**

In [2]:
# cargamos el archivo con los datos
df_raw = pd.read_parquet("../data/data_raw/reservas_hoteles.parquet")

In [3]:
# Hacemos copia del dataframe para trabajar
df = df_raw.copy()

# Cambio columnas con fechas a datetime

In [4]:
# Las columnas de fecha_reserva, inicio_estancia y final_estancia deberían estar en formato fecha. Hacemos una función para transformarlas.
def trans_fecha(lista_col, dataframe):
    for col in lista_col:
        dataframe[col] = pd.to_datetime(dataframe[col], errors="coerce")
    return dataframe

In [5]:
df = trans_fecha(["fecha_reserva", "inicio_estancia", "final_estancia"], df)

# Fechas estancia

In [6]:
# miramos valores únicos de fechas de inicio y final de estancia porque las necesitaremos para buscar eventos en la API
ini_estancia =df["inicio_estancia"].unique()
fin_estancia = df["final_estancia"].unique()

Como las fechas son las mismas, podríamos rellenar toda la columna con el mismo valor.

In [7]:
# Rellenamos las columnas del df con las fechas.
df["inicio_estancia"] = ini_estancia[0]
df["final_estancia"] = fin_estancia[0]

# Ciudades

In [51]:
# Rellenamos toda la columna con Madrid.
df["ciudad"] = "Madrid"

# Duplicados

In [9]:
df.duplicated().sum()

np.int64(98)

In [10]:
# Hay 98 filas totalmente duplicadas, las eliminamos
df = df.drop_duplicates()

# **Transformación**

De cara a cargar los datos en la base de datos vamos a generar unas tablas a partir del df.

In [11]:
# Podemos tener id_hotel duplicados porque más de un cliente puede reservar en el mismo hotel y podemos tener id_cliente duplicados porque el mismo cliente puede haber hecho varias reservas
# lo que no podemos tener repetido es el id_reserva. Lo comprobamos
df["id_reserva"].duplicated().sum()

np.int64(0)

## Ciudades

In [52]:
ciudades_limpio = df["ciudad"].unique()
df_ciudades = pd.DataFrame(ciudades_limpio, columns=["nombre_ciudad"])
df_ciudades

,nombre_ciudad
0,Madrid


## Hoteles

In [13]:
# Cantidad de id_hotel
df["id_hotel"].nunique() == df["nombre_hotel"].nunique()

False

Hay discrepancia en la cantidad de valores únicos. Vamos a generar dos tablas de hoteles, una con los hoteles propios y otra con los de la competencia.

### Hoteles propios

In [14]:
info_propios = df[["id_hotel", "nombre_hotel" , "estrellas", "ciudad", "competencia"]][df["competencia"]==False]

In [22]:
# Comprobamos si hay hoteles con más de un id asignado
conteo_hoteles = info_propios.groupby("nombre_hotel")["id_hotel"].nunique().reset_index()
conteo_hoteles

,nombre_hotel,id_hotel
0,Gran Hotel Madrid,19
1,Hotel Brisas del Mar,19
2,Hotel Camino del Sol,19
3,Hotel Costa Azul,19
4,Hotel Encanto Real,19
5,Hotel Jardines del Rey,19
6,Hotel Las Estrellas,19
7,Hotel Los Almendros,19
8,Hotel Luz de Madrid,19
9,Hotel Maravilla Real,19


Todos los hoteles tienen 19 id asignados, lo que no puede ser correcto. Vamos a asociar un único id_hotel por cada nombre_hotel

In [15]:
# Cremos una lista con los nombres de los hoteles
nombrespropios = list(info_propios["nombre_hotel"].unique())
# Creamos una lista de números seriales según la cantidad de hoteles propios
idspropios = list(range(1, len(nombrespropios)+1))
# Asignamos números seriales a cada elemento. Lo hacemos como un diccionario para poder hacer un mapeo en el df_propios
hoteles_propios = dict(zip(nombrespropios, idspropios))

In [16]:
info_propios["id_hotel"] = info_propios["nombre_hotel"].map(hoteles_propios)

La columna estrellas se supone que se refiere a la clasificación del hotel, por lo que cada hotel debería tener una clasificación única. Lo comprobamos

In [26]:
conteo_estrellas = info_propios.groupby("nombre_hotel")["estrellas"].nunique().reset_index()
conteo_estrellas

,nombre_hotel,estrellas
0,Gran Hotel Madrid,5
1,Hotel Brisas del Mar,5
2,Hotel Camino del Sol,5
3,Hotel Costa Azul,5
4,Hotel Encanto Real,5
5,Hotel Jardines del Rey,5
6,Hotel Las Estrellas,5
7,Hotel Los Almendros,5
8,Hotel Luz de Madrid,5
9,Hotel Maravilla Real,5


Vemos cada hotel tiene 5 clasificaciones distintas, lo que no tendría sentido. Vamos a suponer que se trata de la valoración de los clientes, por lo que tomaremos el valor medio de cada hotel.

In [18]:
# Cambiamos el nombre de la columna
info_propios.rename(columns={"estrellas":"valoracion"}, inplace=True)

In [19]:
# Calculamos la valoracion media por cada nombre de hotel y redondeamos a 1 decimal
df_valoracion_propios = round(info_propios.groupby("nombre_hotel")["valoracion"].mean().reset_index(),1)

In [20]:
def generar_mapa(col1, col2, dataframe):
    """
    Genera un diccionario a partir de los valores de dos columnas de un DataFrame, generando pares clave:valor por cada fila.

    Parámetros:
    col1 (str): Nombre de la primera columna que se usará como claves del diccionario.
    col2 (str): Nombre de la segunda columna que se usará como valores del diccionario.
    dataframe (pandas.DataFrame): El DataFrame que contiene las columnas a mapear.

    Retorna:
    dict: Un diccionario donde las claves son los valores de la columna col1 y los valores son los valores de la columna col2.
    """
    lista_col1 = []
    lista_col2 = []
    for i in dataframe[col1]:
        lista_col1.append(i)
    for f in dataframe[col2]:
        lista_col2.append(f)

    mapa = dict(zip(lista_col1, lista_col2))
    return mapa

In [21]:
# generamos un diccionario con los nombres de los hoteles y su valoración media
valoracion_propios = generar_mapa("nombre_hotel", "valoracion", df_valoracion_propios)

In [ ]:
# Usamos el diccionario para mapear el df de hoteles propios y corregir la columna de estrellas
info_propios["valoracion"] = info_propios["nombre_hotel"].map(valoracion_propios)

In [23]:
# usamos los diccionarios que hemos creado para generar un df con los hoteles propios
df_propios = pd.DataFrame(hoteles_propios.items(), columns=["nombre_hotel", "id_hotel"])
# Creamos las columnas que nos faltan para rellenarlas
df_propios[["valoracion", "ciudad", "competencia"]] = pd.NA
df_propios["valoracion"] = df_propios["nombre_hotel"].map(valoracion_propios)
valor_ciudad = "Madrid"
valor_competencia = False
df_propios["ciudad"] = valor_ciudad
df_propios["competencia"] = valor_competencia

### Hoteles competencia

In [26]:
competencia = df[["id_hotel", "nombre_hotel", "estrellas", "ciudad", "competencia"]][df["competencia"]==True]

In [27]:
# Cambiamos el nombre de la columna estrellas
competencia.rename(columns={"estrellas":"valoracion"}, inplace=True)

In [28]:
ids_competencia = (competencia["id_hotel"].unique()).tolist()

Los datos que faltan los extraemos por scrapeo

In [29]:
info_competencia = pd.read_pickle("../data/data_clean/hoteles_competencia.pkl")

In [30]:
info_competencia["id_hotel"] = ids_competencia
info_competencia["ciudad"] = "Madrid"

In [33]:
df_competencia = info_competencia[["id_hotel", "nombre", "valoracion", "ciudad"]]
competencia = True
df_competencia["competencia"] = competencia
df_competencia.rename(columns={"nombre" : "nombre_hotel"}, inplace = True)

C:\Users\marta\AppData\Local\Temp\ipykernel_12144\4064413440.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_competencia["competencia"] = competencia
C:\Users\marta\AppData\Local\Temp\ipykernel_12144\4064413440.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_competencia.rename(columns={"nombre" : "nombre_hotel"}, inplace = True)


In [35]:
#unimos los dos dataframes de hoteles
df_hoteles = pd.concat([df_propios, df_competencia], axis = 0)

In [36]:
valoracion_hoteles = generar_mapa("nombre_hotel", "valoracion", df_hoteles)
nombres_id_hoteles = generar_mapa("nombre_hotel", "id_hotel", df_hoteles)
nombres_competencia = generar_mapa("id_hotel", "nombre_hotel", df_competencia)
precios_competencia = generar_mapa("nombre", "precio", info_competencia)

## Clientes

Como podemos tener dos clientes con el mismo nombre, tomaremos el mail de cliente como el valor único de cliente. Es posible que tengamos un mismo cliente que haya hecho vrias reservas, por lo que encontrar el mismo id_cliente varias veces tampoco indicaría error. Vamos a comprobar si el número de valores únicos de id_cliente se corresponde con el número de valores únicos de mail. De no ser así, podría indicar un error en la asignación de id_cliente que debemos corregir

In [39]:
df["id_cliente"].nunique() == df["mail"].nunique()

False

Vemos que hay más valores de mail que de id_cliente, lo que sugiere que más de un cliente tiene el mismo id. Vamos a corregir el id de la misma forma que hemos corregido los id_hotel.

In [40]:
df_clientes = df[["id_cliente", "nombre", "apellido", "mail"]]

In [41]:
mails=df_clientes["mail"].unique()

In [42]:
mailcliente=[]
idcliente=[]
for mail in enumerate(mails, start=1):
    mailcliente.append(mail[1])
    idcliente.append(f"cliente_{mail[0]}")
mail_cliente = dict(zip(mailcliente, idcliente))

In [43]:
df_clientes.loc[:, "id_cliente"] = df_clientes["mail"].map(mail_cliente)

In [44]:
df_clientes = df_clientes.drop_duplicates()

In [45]:
df_clientes.duplicated().sum()

np.int64(0)

## Reservas

In [53]:
# Renombramos la columna estrellas por valoracion
df.rename(columns = {"estrellas": "valoracion"}, inplace = True)

In [46]:
# La fecha de reserva para los hoteles de la competencia es la del scrapeo. Lo actualizamos
df.loc[df["competencia"]==True, "fecha_reserva"] = info_competencia["fecha_reserva"][0]
df["id_cliente"] = df["mail"].map(mail_cliente)
df.loc[df["competencia"]==False, "id_hotel"] = df["nombre_hotel"].map(hoteles_propios)
df.loc[df["competencia"] == True, "nombre_hotel"] = df["id_hotel"].map(nombres_competencia)
df.loc[df["competencia"] == True, "precio_noche"] = df["nombre_hotel"].map(precios_competencia)
df["valoracion"] = df["nombre_hotel"].map(valoracion_hoteles)

In [47]:
df_reservas = df[["id_reserva", "fecha_reserva", "inicio_estancia", "final_estancia", "precio_noche", "id_cliente", "id_hotel", "nombre_hotel"]]

In [48]:
df_precios = df_reservas.groupby("id_hotel")["precio_noche"].describe()

In [49]:
# Creamos un diccionario con los precios medios de los hoteles para poder rellenar los valores nulos
precios_medios = dict(df_precios["mean"])
df.loc[df["competencia"]==False, "id_hotel"] = df["nombre_hotel"].map(hoteles_propios)
df_reservas.loc[df_reservas["precio_noche"].isnull(), "precio_noche"] = df_reservas["id_hotel"].map(precios_medios)

In [53]:
#Guardamos los df generados en archivos pickle
df_ciudades.to_pickle("../data/data_clean/ciudades.pkl")
df_hoteles.to_pickle("../data/data_clean/hoteles.pkl")
df_clientes.to_pickle("../data/data_clean/clientes.pkl")
df_reservas.to_pickle("../data/data_clean/reservas.pkl")